In [0]:
from pyspark import SparkConf,SparkContext

In [0]:
#conf = SparkConf().setMaster("local[*]").setAppName("RatingsHistogram")
#sc=SparkConf(conf=conf)

In [0]:
lines=sc.textFile("dbfs:/FileStore/tables/u.data")
lines.take(5)

Out[2]: ['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [0]:
ratings=lines.map(lambda x:x.split()[2])
ratings.take(5)

Out[3]: ['3', '3', '1', '2', '1']

In [0]:
result=ratings.countByValue()
print(result)

defaultdict(<class 'int'>, {'3': 27145, '1': 6110, '2': 11370, '4': 34174, '5': 21201})

In [0]:
import collections

In [0]:
# Movie Ratings Counter 

In [0]:
sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print("%s %i" % (key, value))

1 6110
2 11370
3 27145
4 34174
5 21201

In [0]:
from pyspark.sql import SparkSession,functions as func
from pyspark.sql.types import StructType,StructField,LongType,IntegerType,StringType,FloatType,IntegerType

In [0]:
spark=SparkSession.builder.appName("PopularMovie").getOrCreate()

In [0]:
schema=StructType([StructField("user_id",IntegerType(),True),StructField("movie_id",IntegerType(),True),StructField("rating",IntegerType(),True),StructField("timestamp",LongType(),True)])

In [0]:
new_df=spark.read.option("sep","\t").schema(schema).csv("dbfs:/FileStore/tables/u.data")
new_df.show(5)

+-------+--------+------+---------+
user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
 196| 242| 3|881250949|
 186| 302| 3|891717742|
 22| 377| 1|878887116|
 244| 51| 2|880606923|
 166| 346| 1|886397596|
+-------+--------+------+---------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import col

In [0]:
moviePairs=new_df.alias("new_1").join(new_df.alias("new_2"),col("new_1.user_id")==col("new_2.user_id"),"inner") \
           .select(col("new_1.movie_id"),col("new_1.rating"),col("new_2.movie_id"),col("new_2.rating"))
moviePairs.show(10)

+--------+------+--------+------+
movie_id|rating|movie_id|rating|
+--------+------+--------+------+
 242| 3| 110| 1|
 242| 3| 269| 3|
 242| 3| 845| 4|
 242| 3| 1022| 4|
 242| 3| 173| 2|
 242| 3| 762| 3|
 242| 3| 13| 2|
 242| 3| 153| 5|
 242| 3| 411| 4|
 242| 3| 1007| 4|
+--------+------+--------+------+
only showing top 10 rows

In [0]:
new_df.createOrReplaceTempView("user")
spark.sql("select * from user limit 5").show()

+-------+--------+------+---------+
user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
 196| 242| 3|881250949|
 186| 302| 3|891717742|
 22| 377| 1|878887116|
 244| 51| 2|880606923|
 166| 346| 1|886397596|
+-------+--------+------+---------+

In [0]:
new_df.groupBy("movie_id").count().sort("count",ascending=False).show(10)

+--------+-----+
movie_id|count|
+--------+-----+
 50| 583|
 258| 509|
 100| 508|
 181| 507|
 294| 485|
 286| 481|
 288| 478|
 1| 452|
 300| 431|
 121| 429|
+--------+-----+
only showing top 10 rows

In [0]:
popular_df=new_df.groupBy("movie_id").count().sort("count",ascending=False)
display(popular_df)

movie_id,count
50,583
258,509
100,508
181,507
294,485
286,481
288,478
1,452
300,431
121,429


In [0]:
new_df.groupBy("movie_id").count().orderBy(func.desc("count")).show(10)

+--------+-----+
movie_id|count|
+--------+-----+
 50| 583|
 258| 509|
 100| 508|
 181| 507|
 294| 485|
 286| 481|
 288| 478|
 1| 452|
 300| 431|
 121| 429|
+--------+-----+
only showing top 10 rows

In [0]:
new_df.groupBy("rating").count().show()

+------+-----+
rating|count|
+------+-----+
 1| 6110|
 3|27145|
 5|21201|
 4|34174|
 2|11370|
+------+-----+

In [0]:
rating_df=new_df.groupBy("rating").count()
display(rating_df)

rating,count
1,6110
3,27145
5,21201
4,34174
2,11370


In [0]:
display(rating_df)

rating,count
1,6110
3,27145
5,21201
4,34174
2,11370


In [0]:
display(rating_df)

rating,count
1,6110
3,27145
5,21201
4,34174
2,11370


In [0]:
new_schema=StructType([StructField("movie_id",IntegerType(),True),StructField("movie_names",StringType(),True)])

In [0]:
df2=spark.read.schema(new_schema).option("sep","|").csv("dbfs:/FileStore/tables/u.item")
df2.show(5)

+--------+-----------------+
movie_id| movie_names|
+--------+-----------------+
 1| Toy Story (1995)|
 2| GoldenEye (1995)|
 3|Four Rooms (1995)|
 4|Get Shorty (1995)|
 5| Copycat (1995)|
+--------+-----------------+
only showing top 5 rows

In [0]:
df2.createOrReplaceTempView("movie")
spark.sql("select * from movie limit 5").show()

+--------+-----------------+
movie_id| movie_names|
+--------+-----------------+
 1| Toy Story (1995)|
 2| GoldenEye (1995)|
 3|Four Rooms (1995)|
 4|Get Shorty (1995)|
 5| Copycat (1995)|
+--------+-----------------+

In [0]:
new_df.join(df2,new_df.movie_id==df2.movie_id,"inner").show(10)

+-------+--------+------+---------+--------+--------------------+
user_id|movie_id|rating|timestamp|movie_id| movie_names|
+-------+--------+------+---------+--------+--------------------+
 196| 242| 3|881250949| 242| Kolya (1996)|
 186| 302| 3|891717742| 302|L.A. Confidential...|
 22| 377| 1|878887116| 377| Heavyweights (1994)|
 244| 51| 2|880606923| 51|Legends of the Fa...|
 166| 346| 1|886397596| 346| Jackie Brown (1997)|
 298| 474| 4|884182806| 474|Dr. Strangelove o...|
 115| 265| 2|881171488| 265|Hunt for Red Octo...|
 253| 465| 5|891628467| 465|Jungle Book, The ...|
 305| 451| 3|886324817| 451| Grease (1978)|
 6| 86| 3|883603013| 86|Remains of the Da...|
+-------+--------+------+---------+--------+--------------------+
only showing top 10 rows

In [0]:
spark.sql("select u.*,m.movie_names from user u inner join movie m on u.movie_id==m.movie_id").show(10)

+-------+--------+------+---------+--------------------+
user_id|movie_id|rating|timestamp| movie_names|
+-------+--------+------+---------+--------------------+
 196| 242| 3|881250949| Kolya (1996)|
 186| 302| 3|891717742|L.A. Confidential...|
 22| 377| 1|878887116| Heavyweights (1994)|
 244| 51| 2|880606923|Legends of the Fa...|
 166| 346| 1|886397596| Jackie Brown (1997)|
 298| 474| 4|884182806|Dr. Strangelove o...|
 115| 265| 2|881171488|Hunt for Red Octo...|
 253| 465| 5|891628467|Jungle Book, The ...|
 305| 451| 3|886324817| Grease (1978)|
 6| 86| 3|883603013|Remains of the Da...|
+-------+--------+------+---------+--------------------+
only showing top 10 rows

In [0]:
final_df=spark.sql("select u.user_id,u.movie_id,m.movie_names,u.rating,u.timestamp from user u inner join movie m on u.movie_id==m.movie_id")
final_df.show(10)

+-------+--------+--------------------+------+---------+
user_id|movie_id| movie_names|rating|timestamp|
+-------+--------+--------------------+------+---------+
 196| 242| Kolya (1996)| 3|881250949|
 186| 302|L.A. Confidential...| 3|891717742|
 22| 377| Heavyweights (1994)| 1|878887116|
 244| 51|Legends of the Fa...| 2|880606923|
 166| 346| Jackie Brown (1997)| 1|886397596|
 298| 474|Dr. Strangelove o...| 4|884182806|
 115| 265|Hunt for Red Octo...| 2|881171488|
 253| 465|Jungle Book, The ...| 5|891628467|
 305| 451| Grease (1978)| 3|886324817|
 6| 86|Remains of the Da...| 3|883603013|
+-------+--------+--------------------+------+---------+
only showing top 10 rows

In [0]:
display(final_df)

user_id,movie_id,movie_names,rating,timestamp
196,242,Kolya (1996),3,881250949
186,302,L.A. Confidential (1997),3,891717742
22,377,Heavyweights (1994),1,878887116
244,51,Legends of the Fall (1994),2,880606923
166,346,Jackie Brown (1997),1,886397596
298,474,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),4,884182806
115,265,"Hunt for Red October, The (1990)",2,881171488
253,465,"Jungle Book, The (1994)",5,891628467
305,451,Grease (1978),3,886324817
6,86,"Remains of the Day, The (1993)",3,883603013


In [0]:
count_df=final_df.groupBy("movie_id").count().orderBy(func.desc("count"))
count_df.show(10)

+--------+-----+
movie_id|count|
+--------+-----+
 50| 583|
 258| 509|
 100| 508|
 181| 507|
 294| 485|
 286| 481|
 288| 478|
 1| 452|
 300| 431|
 121| 429|
+--------+-----+
only showing top 10 rows

In [0]:
ucount_df=final_df.groupBy("user_id").count().orderBy(func.desc("count"))
ucount_df.show(10)

+-------+-----+
user_id|count|
+-------+-----+
 405| 737|
 655| 685|
 13| 636|
 450| 540|
 276| 518|
 416| 493|
 537| 490|
 303| 484|
 234| 480|
 393| 448|
+-------+-----+
only showing top 10 rows

In [0]:
final_df.createOrReplaceTempView("finalmovie")
count_df.createOrReplaceTempView("moviecount")
ucount_df.createOrReplaceTempView("usercount")

In [0]:
spark.sql("select f.user_id,f.movie_id,m.count,f.movie_names,f.rating,f.timestamp from finalmovie f inner join moviecount m on f.movie_id==m.movie_id").show(10)

+-------+--------+-----+--------------------+------+---------+
user_id|movie_id|count| movie_names|rating|timestamp|
+-------+--------+-----+--------------------+------+---------+
 250| 496| 231|It's a Wonderful ...| 4|878090499|
 293| 471| 221|Courage Under Fir...| 3|888904884|
 305| 471| 221|Courage Under Fir...| 4|886323648|
 203| 471| 221|Courage Under Fir...| 4|880434463|
 6| 463| 71|Secret of Roan In...| 4|883601713|
 124| 496| 231|It's a Wonderful ...| 1|890286933|
 186| 148| 128|Ghost and the Dar...| 4|891719774|
 201| 148| 128|Ghost and the Dar...| 1|884140751|
 23| 463| 71|Secret of Roan In...| 4|874785843|
 94| 471| 221|Courage Under Fir...| 4|891721642|
+-------+--------+-----+--------------------+------+---------+
only showing top 10 rows

In [0]:
df3=spark.sql("select f.user_id,f.movie_id,m.count,f.movie_names,f.rating,f.timestamp from finalmovie f inner join moviecount m on f.movie_id==m.movie_id")
df3.show(5)

+-------+--------+-----+--------------------+------+---------+
user_id|movie_id|count| movie_names|rating|timestamp|
+-------+--------+-----+--------------------+------+---------+
 250| 496| 231|It's a Wonderful ...| 4|878090499|
 293| 471| 221|Courage Under Fir...| 3|888904884|
 305| 471| 221|Courage Under Fir...| 4|886323648|
 203| 471| 221|Courage Under Fir...| 4|880434463|
 6| 463| 71|Secret of Roan In...| 4|883601713|
+-------+--------+-----+--------------------+------+---------+
only showing top 5 rows

In [0]:
df4=df3.withColumnRenamed('count','movie_count')
df4.show(5)

+-------+--------+-----------+--------------------+------+---------+
user_id|movie_id|movie_count| movie_names|rating|timestamp|
+-------+--------+-----------+--------------------+------+---------+
 250| 496| 231|It's a Wonderful ...| 4|878090499|
 293| 471| 221|Courage Under Fir...| 3|888904884|
 305| 471| 221|Courage Under Fir...| 4|886323648|
 203| 471| 221|Courage Under Fir...| 4|880434463|
 6| 463| 71|Secret of Roan In...| 4|883601713|
+-------+--------+-----------+--------------------+------+---------+
only showing top 5 rows

In [0]:
df4.createOrReplaceTempView("temp_table")

In [0]:
spark.sql("select d.user_id,c.count,d.movie_id,d.movie_count,d.movie_names,d.rating,d.timestamp from temp_table d inner join usercount c on d.user_id==c.user_id").show(5)

+-------+-----+--------+-----------+--------------------+------+---------+
user_id|count|movie_id|movie_count| movie_names|rating|timestamp|
+-------+-----+--------+-----------+--------------------+------+---------+
 250| 122| 496| 231|It's a Wonderful ...| 4|878090499|
 293| 388| 471| 221|Courage Under Fir...| 3|888904884|
 305| 222| 471| 221|Courage Under Fir...| 4|886323648|
 203| 43| 471| 221|Courage Under Fir...| 4|880434463|
 6| 211| 463| 71|Secret of Roan In...| 4|883601713|
+-------+-----+--------+-----------+--------------------+------+---------+
only showing top 5 rows

In [0]:
df5=spark.sql("select d.user_id,c.count,d.movie_id,d.movie_count,d.movie_names,d.rating,d.timestamp from temp_table d inner join usercount c on d.user_id==c.user_id")
df5.show(10)

+-------+-----+--------+-----------+--------------------+------+---------+
user_id|count|movie_id|movie_count| movie_names|rating|timestamp|
+-------+-----+--------+-----------+--------------------+------+---------+
 496| 129| 496| 231|It's a Wonderful ...| 1|876066424|
 148| 65| 496| 231|It's a Wonderful ...| 3|877015066|
 833| 267| 540| 43| Money Train (1995)| 1|875224687|
 463| 133| 243| 132|Jungle2Jungle (1997)| 1|877384970|
 496| 129| 1139| 33| Hackers (1995)| 2|876073882|
 833| 267| 1143| 15| Hard Eight (1996)| 4|887158946|
 833| 267| 451| 170| Grease (1978)| 1|875134016|
 463| 133| 137| 171| Big Night (1996)| 2|877385237|
 471| 31| 588| 202|Beauty and the Be...| 1|889827881|
 148| 65| 588| 202|Beauty and the Be...| 4|877399018|
+-------+-----+--------+-----------+--------------------+------+---------+
only showing top 10 rows

In [0]:
df5=df5.withColumnRenamed('count','user_count')
df5.show(10)

+-------+----------+--------+-----------+--------------------+------+---------+
user_id|user_count|movie_id|movie_count| movie_names|rating|timestamp|
+-------+----------+--------+-----------+--------------------+------+---------+
 250| 122| 496| 231|It's a Wonderful ...| 4|878090499|
 293| 388| 471| 221|Courage Under Fir...| 3|888904884|
 305| 222| 471| 221|Courage Under Fir...| 4|886323648|
 203| 43| 471| 221|Courage Under Fir...| 4|880434463|
 6| 211| 463| 71|Secret of Roan In...| 4|883601713|
 124| 24| 496| 231|It's a Wonderful ...| 1|890286933|
 186| 92| 148| 128|Ghost and the Dar...| 4|891719774|
 201| 386| 148| 128|Ghost and the Dar...| 1|884140751|
 23| 151| 463| 71|Secret of Roan In...| 4|874785843|
 94| 400| 471| 221|Courage Under Fir...| 4|891721642|
+-------+----------+--------+-----------+--------------------+------+---------+
only showing top 10 rows

In [0]:
display(df5)

user_id,user_count,movie_id,movie_count,movie_names,rating,timestamp
691,32,496,231,It's a Wonderful Life (1946),5,875543025
654,147,496,231,It's a Wonderful Life (1946),4,887864230
479,202,496,231,It's a Wonderful Life (1946),3,879461084
890,119,496,231,It's a Wonderful Life (1946),5,882916460
731,83,496,231,It's a Wonderful Life (1946),5,886179040
665,142,496,231,It's a Wonderful Life (1946),3,884294200
929,49,496,231,It's a Wonderful Life (1946),3,879640256
426,101,496,231,It's a Wonderful Life (1946),3,879442841
698,127,496,231,It's a Wonderful Life (1946),3,886366690
807,204,496,231,It's a Wonderful Life (1946),5,892528918
